In [ ]:
import numpy as np
import slmsuite
from matplotlib import pyplot as plt
from slmsuite.holography.algorithms import Hologram
from tools.IonChainTools import *
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import scipy.constants as con
from PIL import Image
from math import sin, pi as π
from slmsuite.holography.toolbox import _process_grid
import cv2

In [ ]:
#loading the slm-200
from slmsuite.hardware.slms.santec import Santec
Santec.info(verbose=True)
#set wavelength to the correct value
slm = Santec(slm_number=1, display_number=2, wav_um=.730, settle_time_s=.3)

In [ ]:
# load the camera
from slmsuite.hardware.cameras.alliedvision import AlliedVision
AlliedVision.info(verbose=True)
#find the camera we want,then choose it
cam = AlliedVision(serial="0245P", verbose=True)
from slmsuite.hardware.cameraslms import FourierSLM
fs = FourierSLM(cam, slm)

In [ ]:
#load vendor-provided wavefront calibraiton file
slm.load_vendor_phase_correction(
 file_path='e:\A_my project in Seattle\project\A_intensity generator\slm_initiate\Wavefront_correction_Data_adjust(730nm).csv', 
 smooth=True
)

In [ ]:
#Fourier calibration
#Remember everytime we move the camera or change the setup we need to do the Fourier calibration again
#When I'm doing the fourier caliration, I put an OD2 and tune the current value to be 24mA for reference. For pratical use, if you find the calibration works not well, please close the camera with demand" cam.close" and then using the "vimba viewer" to find the best exposure time and current value so that the camera can detect the array dots

cam.set_exposure(100e-6) # set the exposure time for the camera

fs.fourier_calibrate(
    array_shape=[10, 10],           # Size of the calibration grid (Nx, Ny) [knm]
    array_pitch=[18, 18],           # Pitch of the calibration grid (x, y) [knm]
    plot=True
)

In [ ]:
#Remember to save the calibration data after calibration each time and next time you only need to load the data

# fs.save_fourier_calibration()
fs.load_fourier_calibration()

In [ ]:
# functions for caculating the ions' position on the camera
mCa40 = con.atomic_mass*39.9626

def ion_position_act(N,v): #N is the number of the ions, v is the trap frequency
    return calcPositions(N)*lengthScale(v)

def ion_position_cam(N,v,pixel_size,magnititude = 200):
    position = ion_position_act(N,v)*magnititude//pixel_size[0]
    return position

# function for generating blaze grating phase
def blaze_grating_phase(grid, vector=(0, 0),offset=0,):
    (x_grid, y_grid) = _process_grid(grid)

    # Optimize phase construction based on context.
    if vector[0] == 0 and vector[1] == 0:
        return np.zeros_like(x_grid) + offset
    elif vector[0] == 0:
        return 2 * np.pi * (vector[1] * y_grid) + offset
    elif vector[1] == 0:
        return 2 * np.pi * (vector[0] * x_grid) + offset
    else:
        return 2 * np.pi * (vector[0] * x_grid + vector[1] * y_grid) + offset

# function for generating blaze grating phase
def vortex_pattern(grid,l):
    (x_grid, y_grid) = _process_grid(grid)
    
    theta = np.arctan2(y_grid,x_grid)
    phase = (l * theta) % (2 * np.pi)
    
    return phase

# function for generating spatial filter
def create_spatial_filter(grid):
    (x_grid, y_grid) = _process_grid(grid)
    radius = np.sqrt(x_grid[0,0]**2+y_grid[0,0]**2)/ 4
    filter_mask = x_grid**2 + y_grid**2 <= radius**2
    return filter_mask 

# function for generating Sinusoidal grating phase
def Sinusoidal_Phase_Grating(grid, vector=(0, 0), m=1,k=1,offset = 0):
    (x_grid, y_grid) = _process_grid(grid)
    if vector[0] == 0 and vector[1] == 0:
        return np.zeros_like(x_grid) + offset
    elif vector[0] == 0:
        return m/2*np.sin(2 * np.pi * k*(vector[1] * y_grid) + offset)
    elif vector[1] == 0:
        return m/2*np.sin(2 * np.pi * k*(vector[0] * x_grid) + offset)
    else:
        return m/2*np.sin(2 * np.pi * k*(vector[0] * x_grid + vector[1] * y_grid) + offset)
    
# Using the grating phase to excite the ions
def excite_generator(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1,m=4.71):
    
    position = ion_position_cam(N,v,pixel_size) # calculate ions' position in x-axis
    dy = pixels[0]//8   #displacement of the y-axis
    
    position_cam = np.column_stack((position+pixels[1]//2,np.full(position.shape, pixels[0]//2+dy))) #positions in the camera
    
    vector = []
    for i, pos in enumerate(position_cam):
        vector.append(fs.ijcam_to_kxyslm(pos))
        
    blaze_phase = []
    for i, vec in enumerate(vector):
        # blaze_phase.append(Sinusoidal_Phase_Grating(grid=slm, vector=vec,m=m,k=k,offset = offset))
        blaze_phase.append(blaze_grating_phase(grid=slm, vector=vec,offset = offset))
    
    result_shape = blaze_phase[0].shape  
    new_holo = np.zeros(result_shape, dtype=np.complex128)

    for i in ions:
        new_holo += np.exp(1j * blaze_phase[i])

    phase = np.angle(new_holo)
    
    #adding a spatial filter
    if filter == 1:
        spatial_filter = create_spatial_filter(grid = slm)
        new_phase = spatial_filter * phase
        return new_phase
    else: return phase

#The function for writing the phase on the SLM and plot the image from the camera
def slm_write(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1,m=1,plot = 0,cal = 0):  
    phase  = excite_generator(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1)#m=4.71,c=0.7
    slm.write(phase)
    
    if plot == 1 or cal ==1:
        cam.set_exposure(100e-6)
        cam.flush()
        img = cam.get_image()
        position = ion_position_cam(N,v,pixel_size)
        dy = pixels[0]//8
        position_cam = np.column_stack((position+pixels[1]//2,np.full(position.shape, pixels[0]//2+dy)))
    
    if plot == 1:         #plot the image from the camera with circles enclosed the spots
        
        for i, pos in enumerate(position_cam):
        cv2.circle(img, (int(pos[0]), int(pos[1])), 60, (255, 0, 0), 2)
        plt.imshow(img)
        
    if cal ==1 :          #calculate the intensity on different ions
        inten = []
        inten.append(np.sum(img[1944//2-500-60:1944//2-500+60,2592//2-60:2592//2+60])/255.0)
        for i, pos in enumerate(position_cam):
            inten.append(np.sum(img[int(pos[1])-60:int(pos[1])+60,int(pos[0])-60:int(pos[0])+60])/255.0)
        return inten
          

In [ ]:
#the camera we are using has pixels:1944*2592, with pixel_size of 2.2um*2.2um

N = 6  # The number of the ions

v = 2*π*1e6   # trap frequency  
                 
pixel_size = np.array([2.2e-6,2.2e-6]) 
pixels = np.array([1944,2592])

ions = np.array([0,1,2,3,4,5]) #Here you determine the ions you want to excite.

slm_write(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1,m=1,plot = 0,cal = 0)


In [ ]:
#calculate the fidelity
result = slm_write(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1,m=1,plot = 0,cal = 1)
plt.scatter(range(7),result, c='blue', marker='o', label='data')
plt.xlabel('ions')
plt.ylabel('intensity')
# plt.ylim(1250, 2250)
# plt.yticks(np.arange(1250,2250,125))
plt.legend()
plt.show()

from numpy import sum, sqrt, max
import numpy as np
intensity = result[1:]
background = result[0]

inten = np.array([ i- background for i in intensity])
inten[inten < 0] = 0
all = range(N)
undesire = [ i for i in all if i not in ions]
Omega_desire = sqrt(inten[ions])
Omega_undesire = sqrt(inten[undesire])
F = sum(Omega_desire)/(len(ions)*max(Omega_desire)) *(1-sum(Omega_undesire)/(len(undesire)*max(Omega_desire)))

print(F)